# COGS 118B - Final Project

# Insert title here

## Group members

- Daniel Kong
- Jonathan Ito
- Uri Reyes
- Larry Nguyen
- Fatima

# Abstract 
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize: 
- what your goal/problem is
- what the data used represents 
- the solution/what you did
- major results you came up with (mention how results are measured) 

__NB:__ this final project form is much more report-like than the proposal and the checkpoint. Think in terms of writing a paper with bits of code in the middle to make the plots/tables

# Background

Fill in the background and discuss the kind of prior work that has gone on in this research area here. **Use inline citation** to specify which references support which statements.  You can do that through HTML footnotes (demonstrated here). I used to reccommend Markdown footnotes (google is your friend) because they are simpler but recently I have had some problems with them working for me whereas HTML ones always work so far. So use the method that works for you, but do use inline citations.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds<a name="lorenz"></a>[<sup>[1]</sup>](#lorenznote). Use a minimum of 2 or 3 citations, but we prefer more <a name="admonish"></a>[<sup>[2]</sup>](#admonishnote). You need enough citations to fully explain and back up important facts. 

Remeber you are trying to explain why someone would want to answer your question or why your hypothesis is in the form that you've stated. 

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

# Data

Detail how/where you obtained the data and cleaned it (if necessary)

If the data cleaning process is very long (e.g., elaborate text processing) consider describing it briefly here in text, and moving the actual clearning process to another notebook in your repo (include a link here!).  The idea behind this approach: this is a report, and if you blow up the flow of the report to include a lot of code it makes it hard to read.

Please give the following infomration for each dataset you are using
- link/reference to obtain it
- description of the size of the dataset (# of variables, # of observations)
- what an observation consists of
- what some critical variables are, how they are represented
- any special handling, transformations, cleaning, etc you have done should be demonstrated here!


# Proposed Solution

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared. 

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

# Results

You may have done tons of work on this. Not all of it belongs here. 

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Maybe you include a learning curve to show whether you have enough data to do train/validate/test split or have to go to k-folds or LOOCV or ???

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5 

Maybe you do model selection again, but using a different kind of metric than before?



# Cleaning and Standardizing the Data

- Making sure everything is uniform and up to standard for analysis

In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Some useful imports

import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
from sklearn.cluster import KMeans
import pandas as pd
import string

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
import nltk.data
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/danielkong/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielkong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data = pd.read_csv("spotify_songs.csv")


Drop the columns that are not necessary to analyze


In [4]:
cols_to_drop = ["track_id", "playlist_id", "track_album_id"]
data = data.drop(cols_to_drop, axis = 1)

In [5]:
type(data['mode'].iloc[0])

numpy.int64

### Standardize the Data

Standardizing the data in our case is making the data easier to read and the dataset more understandable. Our methods of standardizing includes:

- Change the length of the songs from milliseconds into format that is in  Minutes: seconds
- Change the mode values so that 1 reads as "major" and 0 reads as "minor" as the dataset creators describe
- Change the key values of tracks to strings of their actual keys rather than integers that represent the keys

In [6]:
type(data['key'].iloc[0])

numpy.int64

In [7]:
def standardize_duration(duration):
    total_seconds = duration / 1000
    minutes = int(total_seconds / 60)
    seconds = int(total_seconds % 60)
    str_seconds = str(seconds)
    if len(str_seconds) < 2:
        str_seconds = "0" + str_seconds
    return str(minutes) + ":" + str_seconds

def standardize_mode(mode):
    if (mode == 1):
        return 'Major'
    else:
        return 'minor'

def standardize_keys(key):
    key_dict = {0: 'C', 1: 'C sharp / D minor', 2: 'D', 3: 'E minor / D sharp', 4: 'E', 5: 'F', 6: 'F Sharp / G Flat', 7: 'G', 8: 'G Sharp / A Flat', 9: 'A', 10: 'A Sharp / B Flat', 11: 'B'}
    return key_dict[key]

In [8]:
data = data.rename(columns={"duration_ms": "track_length"})
data['track_length'] = data['track_length'].apply(standardize_duration)
data['key'] = data['key'].apply(standardize_keys)

In [9]:
data['mode'] = data['mode'].apply(standardize_mode)

In [10]:
#Use this cell to apply the standardize_key() function to the respective column

Change it to only songs in English since we are utilizing Natural Language Processing. Then drop the column since all the songs are in the same language, it is not necessary to keep

In [11]:
data = data[data['language'] == 'en']
data = data.drop(columns = ['language'])
data

,track_name,track_artist,lyrics,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_length
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,Love & Loss,2017-11-21,Hard Rock Workout,rock,hard rock,0.303,...,A,-4.739,Major,0.0442,0.01170,0.009940,0.3470,0.404,135.225,6:13
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",r&b,new jack swing,0.845,...,F Sharp / G Flat,-7.504,minor,0.2160,0.00432,0.007230,0.4890,0.650,111.904,4:22
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,CeeLo's Magic Moment,2012-10-29,Christmas Soul,r&b,neo soul,0.425,...,F,-5.819,minor,0.0341,0.68900,0.000000,0.0664,0.405,118.593,4:03
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,pop,dance pop,0.760,...,A,-1.993,Major,0.0409,0.03700,0.000000,0.1380,0.240,130.018,3:13
5,Soldier,James TW,"Hold your breath, don't look down, keep trying...",70,Chapters,2019-04-26,urban contemporary,r&b,urban contemporary,0.496,...,F Sharp / G Flat,-6.157,Major,0.0550,0.28000,0.000000,0.0975,0.305,147.764,3:44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18448,Some Way,NAV,"Yeah, nah, nah, nah, nah, nah, nah, nah, nah N...",72,NAV,2017-02-24,PROJECT: Contemporary,r&b,urban contemporary,0.744,...,C,-6.103,Major,0.3510,0.10100,0.000000,0.0919,0.340,154.962,2:59
18449,Rising Like The Sun - Radio Mix,Qulinez,"Caught up in such a head rush, wide-eyed latel...",0,Rising Like The Sun,2014-03-24,♥ EDM LOVE 2020,edm,progressive electro house,0.479,...,A Sharp / B Flat,-5.778,minor,0.0878,0.00555,0.000000,0.3350,0.211,128.012,3:28
18450,Anaconda,Nicki Minaj,"My anaconda don't, my anaconda don't My anacon...",49,The Pinkprint (Deluxe Edition),2014-12-15,10er Playlist,pop,electropop,0.963,...,D,-6.224,Major,0.1800,0.06730,0.000006,0.2140,0.647,129.990,4:20
18451,Bound,Ponderosa Twins Plus One,"Bound, bound Bound, bound Bound to fall in lov...",40,2+2+1 = (Digitally Remastered),2013-07-09,Sexy Soul 2020,r&b,neo soul,0.458,...,F,-6.457,minor,0.0270,0.71500,0.000428,0.1150,0.657,142.218,3:11


Let's just create a dataframe of lyrics and song names to separate the data in an organized manner to perform text analysis on the lyrics

In [12]:
lyrics = data[['track_name', 'lyrics']]
lyrics

,track_name,lyrics
1,I Feel Alive,"The trees, are singing in the wind The sky blu..."
2,Poison,"NA Yeah, Spyderman and Freeze in full effect U..."
3,Baby It's Cold Outside (feat. Christina Aguilera),I really can't stay Baby it's cold outside I'v...
4,Dumb Litty,Get up out of my business You don't keep me fr...
5,Soldier,"Hold your breath, don't look down, keep trying..."
...,...,...
18448,Some Way,"Yeah, nah, nah, nah, nah, nah, nah, nah, nah N..."
18449,Rising Like The Sun - Radio Mix,"Caught up in such a head rush, wide-eyed latel..."
18450,Anaconda,"My anaconda don't, my anaconda don't My anacon..."
18451,Bound,"Bound, bound Bound, bound Bound to fall in lov..."


Let's add a column with the unique words of each song. In other words, each song's value for "unique words" is a list of each unique, non-repeated word in the song. 

In [13]:
type(lyrics['lyrics'].iloc[0])

str

In [25]:
def unique_words(words):
    lyrics_list = words.split(" ")
    unique = []
    for word in lyrics_list:
        if ',' in word:
            word = word[:-1]
        if word.lower() not in unique:
            unique.append(word.lower())

    return unique

In [26]:
unique = lyrics['lyrics'].apply(unique_words)

lyrics['unique_words'] = unique
lyrics

/var/folders/wz/qrswb92s7jn3vl5zln9bllfw0000gn/T/ipykernel_34774/2635694480.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyrics['unique_words'] = unique


,track_name,lyrics,unique_words
1,I Feel Alive,"The trees, are singing in the wind The sky blu...","[the, trees, are, singing, in, wind, sky, blue..."
2,Poison,"NA Yeah, Spyderman and Freeze in full effect U...","[na, yeah, spyderman, and, freeze, in, full, e..."
3,Baby It's Cold Outside (feat. Christina Aguilera),I really can't stay Baby it's cold outside I'v...,"[i, really, can't, stay, baby, it's, cold, out..."
4,Dumb Litty,Get up out of my business You don't keep me fr...,"[get, up, out, of, my, business, you, don't, k..."
5,Soldier,"Hold your breath, don't look down, keep trying...","[hold, your, breath, don't, look, down, keep, ..."
...,...,...,...
18448,Some Way,"Yeah, nah, nah, nah, nah, nah, nah, nah, nah N...","[yeah, nah, if, i'ma, talk, keep, it, real, me..."
18449,Rising Like The Sun - Radio Mix,"Caught up in such a head rush, wide-eyed latel...","[caught, up, in, such, a, head, rush, wide-eye..."
18450,Anaconda,"My anaconda don't, my anaconda don't My anacon...","[my, anaconda, don't, want, none, unless, you,..."
18451,Bound,"Bound, bound Bound, bound Bound to fall in lov...","[bound, to, fall, in, love, you, started, with..."


Now let's remove our stop words. A lot of the lyrics are different so we shall start with some basic stop words such as 'and', 'an', 'the', and other basic words used in almost every english sentence

In [27]:
custom_stop_words = ['and', 'an', 'of', 'to', 'in', 'of', 'with', 'for', 'as', 'at', 'that', 'this', 'is', 'are', 'was', 'were', 'be', 
                    'would', 'will', 'could', 'should', 'a', 'by']

for index, row in lyrics.iterrows():
    unique_words = row['unique_words']
    no_stop_words = [word for word in unique_words if word not in custom_stop_words]
    lyrics.at[index, 'unique_words'] = no_stop_words

lyrics

,track_name,lyrics,unique_words
1,I Feel Alive,"The trees, are singing in the wind The sky blu...","[the, trees, singing, wind, sky, blue, only, i..."
2,Poison,"NA Yeah, Spyderman and Freeze in full effect U...","[na, yeah, spyderman, freeze, full, effect, uh..."
3,Baby It's Cold Outside (feat. Christina Aguilera),I really can't stay Baby it's cold outside I'v...,"[i, really, can't, stay, baby, it's, cold, out..."
4,Dumb Litty,Get up out of my business You don't keep me fr...,"[get, up, out, my, business, you, don't, keep,..."
5,Soldier,"Hold your breath, don't look down, keep trying...","[hold, your, breath, don't, look, down, keep, ..."
...,...,...,...
18448,Some Way,"Yeah, nah, nah, nah, nah, nah, nah, nah, nah N...","[yeah, nah, if, i'ma, talk, keep, it, real, me..."
18449,Rising Like The Sun - Radio Mix,"Caught up in such a head rush, wide-eyed latel...","[caught, up, such, head, rush, wide-eyed, late..."
18450,Anaconda,"My anaconda don't, my anaconda don't My anacon...","[my, anaconda, don't, want, none, unless, you,..."
18451,Bound,"Bound, bound Bound, bound Bound to fall in lov...","[bound, fall, love, you, started, mere, hello,..."


# Discussion

### Interpreting the result

OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.

### Limitations

Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   

### Ethics & Privacy

If your project has obvious potential concerns with ethics or data privacy discuss that here.  Almost every ML project put into production can have ethical implications if you use your imagination. Use your imagination.

Even if you can't come up with an obvious ethical concern that should be addressed, you should know that a large number of ML projects that go into producation have unintended consequences and ethical problems once in production. How will your team address these issues?

Consider a tool to help you address the potential issues such as https://deon.drivendata.org

### Conclusion

Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.

# Footnotes
<a name="lorenznote"></a>1.[^](#lorenz): Lorenz, T. (9 Dec 2021) Birds Aren’t Real, or Are They? Inside a Gen Z Conspiracy Theory. *The New York Times*. https://www.nytimes.com/2021/12/09/technology/birds-arent-real-gen-z-misinformation.html<br> 
<a name="admonishnote"></a>2.[^](#admonish): Also refs should be important to the background, not some randomly chosen vaguely related stuff. Include a web link if possible in refs as above.<br>
<a name="sotanote"></a>3.[^](#sota): Perhaps the current state of the art solution such as you see on [Papers with code](https://paperswithcode.com/sota). Or maybe not SOTA, but rather a standard textbook/Kaggle solution to this kind of problem
